In [3]:
class ToDo:

    progress = 0  # class-level variable

    def __init__(self, description, status):
        self.description = description
        self.is_done = status

    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            ToDo.progress = ToDo.progress + 1
        self.is_done = True
        
    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            ToDo.progress = ToDo.progress - 1
        self.is_done = False
        
class Deadline(ToDo):
    
    progress = 0
    
    def __init__(self, description, status, deadline):
        self.deadline = deadline
        ToDo.__init__(self, description, status)
        
    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            Deadline.progress = Deadline.progress + 1
        self.is_done = True
        
    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            Deadline.progress = Deadline.progress - 1
        self.is_done = False
        
        
items = []
items.append(ToDo("Read book", False))
items.append(ToDo("borrow book", False))
items.append(Deadline("Make coffee", False, "Monday"))

def get_current_progress():
    status = {'Todo': 0, 'Deadline': 0}
    tFlag = 0
    dFlag = 0
    
    for obj in items:
        if isinstance(obj, Deadline) and dFlag == 0:
            print("added to deadline count\n")
            status['Deadline'] = obj.progress
            dFlag = 1
        elif tFlag == 0:
            status['Todo'] = obj.progress
            tFlag = 1
    print(">>> Progress for this session: todos {} deadlines {}".format(status['Todo'], status['Deadline']))

items[0].mark_as_done()
get_current_progress()
items[2].mark_as_done()
get_current_progress()
items[2].mark_as_pending()
get_current_progress()

added to deadline count

>>> Progress for this session: todos 1 deadlines 0
added to deadline count

>>> Progress for this session: todos 1 deadlines 1
added to deadline count

>>> Progress for this session: todos 1 deadlines 0


In [71]:
class ToDo:
    
    progress = 0 # class-level variable
    TYPE_KEY = 'T'

    def __init__(self, description, status):
        self.description = description
        self.is_done = status
        
    def __str__(self):        
        return self.__status_as_icon().center(6) + ' ' + '| ' + ' '.center(5) \
    + '| ' + self.description.ljust(27) + ' ' + '| ' + '-'

    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            ToDo.progress = ToDo.progress + 1
        self.is_done = True

    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            if ToDo.progress != 0:
                ToDo.progress = ToDo.progress - 1
        self.is_done = False

    def __status_as_icon(self):
        return 'X' if self.is_done else '-'
    
    def as_csv(self):
        """ Return the details of todo object as a list,
        suitable to be stored in a csv file.
        """
        return [self.TYPE_KEY, self.description, 'done' if self.is_done else 'pending']
    
    def as_item(self):
        """ Return the name of the item as a string."""
        return self.description
    
class Deadline (ToDo):
    
    progress = 0 # class-level variable
    TYPE_KEY = 'D'

    def __init__(self, description, status, by):
        super().__init__(description, status)
        self.by = by        
    
    def __str__(self):
        s = super().__str__()
        return s[:-1] + self.by
        
    def as_csv(self):
        c = super().as_csv()
        c.append(self.by)
        return c
        
    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            Deadline.progress = Deadline.progress + 1
        self.is_done = True
        
    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            if Deadline.progress != 0:
                Deadline.progress = Deadline.progress - 1
        self.is_done = False

In [73]:
class Error(Exception):
    """Base class for other exceptions"""
    pass

class InvalidInputError(Error):
    """Raised when the input is blank"""
    pass

In [76]:
import datetime
from tkinter import *

import csv
from io import StringIO

import sys

class GUI:

    def __init__(self, task_manager):
        self.task_manager = task_manager
        self.window = Tk()
        self.window.geometry('800x700')  # set Window size
        self.window.title('Monty')  # set Window title

        self.input_box = Entry(self.window)  # create an input box
        self.input_box.pack(padx=5, pady=5, fill='x')  # make the input box fill the width of the Window
        self.input_box.bind('<Return>', self.command_entered)  # bind the command_entered function to the Enter key
        self.input_box.focus()  # set focus to the input box

        # add a text area to show the chat history
        self.history_area = Text(self.window, width="50")
        self.history_area.pack(padx=5, pady=5, side=LEFT, fill="y")
        self.output_font = ('Courier New', 12)
        self.history_area.tag_configure('error_format', foreground='red', font=self.output_font)
        self.history_area.tag_configure('success_format', foreground='green', font=self.output_font)
        self.history_area.tag_configure('normal_format', font=self.output_font)

        # add a text area to show the list of tasks
        self.list_area = Text(self.window)
        self.list_area.pack(padx=5, pady=5, side=RIGHT, fill="both")
        self.list_area.tag_configure('normal_format',  font=self.output_font)
        self.list_area.tag_configure('pending_format', foreground='red', font=self.output_font)
        self.list_area.tag_configure('done_format', foreground='green', font=self.output_font)

        # show the welcome message and the list of tasks
        self.update_chat_history('start', 'Welcome to Monty!', 'success_format')
        self.update_task_list(self.task_manager.items)

    def update_chat_history(self, command, response, status_format):
        """
        status_format: indicates which color to use for the status message
          can be 'error_format', 'success_format', or 'normal_format'
        """
        current_time = datetime.datetime.now().strftime("%H:%M:%S")
        self.history_area.insert(1.0, '-' * 40 + '\n', 'normal_format')
        self.history_area.insert(1.0, '>>> ' + response + '\n', status_format)
        self.history_area.insert(1.0, 'You said: ' + command + '\n', 'normal_format')
        self.history_area.insert(1.0, current_time + '\n', 'normal_format')

    def update_task_list(self, tasks):
        self.list_area.delete('1.0', END)  # clear the list area
        for i, task in enumerate(tasks):
            if task.is_done:
                icon = 'X'
                output_format = 'done_format'
            else:
                icon = '-'
                output_format = 'pending_format'
                
            if isinstance(task, Deadline):
                self.list_area.insert(END, icon + ' ' + str(i+1) + ' ' 
                                  + task.description + ' ' + task.by + '\n', output_format)
            else:
                self.list_area.insert(END, icon + ' ' + str(i+1) + ' ' 
                                  + task.description + '\n', output_format)

    def clear_input_box(self):
        self.input_box.delete(0, END)
    
    def command_entered(self, event):
        command = None
        try:
            command = self.input_box.get().strip()
            if command.lower() == 'exit':
                sys.exit()
            output = self.task_manager.execute_command(command)
            self.update_chat_history(command, output, 'success_format')
            self.update_task_list(self.task_manager.items)
            self.clear_input_box()
        except Exception as e:
            self.update_chat_history(command, str(e) + '\n' + self.task_manager.get_help(), 'error_format')

    def start(self):
        self.window.mainloop()

class TaskManager:
    
    items = []
    filename = 'monty7.csv'
    
    def __init__(self):
        self.load_data()
        self.items = TaskManager.items
    
    def get_help(self):
        return 'TODO help message:\n...'
    
    def load_data(self):
        TaskManager.__create_file_if_missing(self)
        with open(self.filename, 'r') as csvfile:
            file_handler = csv.reader(csvfile)
            for row in file_handler:
                if not row:
                    continue
                self.__load_item_from_csv_line(row)
            return

    def __create_file_if_missing(self):
        open(self.filename, 'a').close()
        
    def __load_item_from_csv_line(self, row):
        if row[0] == 'T':
            self.items.append(ToDo(row[1], True if row[2] == 'True' else False))
        elif row[0] == 'D':
            self.items.append(Deadline(row[1], True if row[2] == 'True' else False, row[3]))
        return
    
    def add_item(self, user_input):
        command_parts = user_input.strip().split(' ', 1)
        try:
            self.items.append(ToDo(command_parts[1], False))
            return ("New item: " + "'" + command_parts[1] + "'" + " added")
        except IndexError:
                raise IndexError("INPUT: todo \"task\"")
        
    def add_deadline_item(self, user_input):
        command_parts = user_input.strip().split(' ', 1)
        try:
            due = command_parts[1].partition("by:")[2].strip()
            task = command_parts[1].partition("by:")[0].strip()
            if due == "" or task == "":
                raise InvalidInputError
            self.items.append(Deadline(task, False, due))
            return ("New item: " + "'" + task + "'" + " added. " + "Deadline: " + "'" + due + "'")
        except InvalidInputError:
            raise InvalidInputError("INPUT: deadline \"task\" by: \"due date\"")
        except IndexError:
            raise IndexError("No deadline task provided")
            
    def mark_item_as_done(self, user_input):
        index_as_string = user_input[5:].strip()
        index_to_remove = self.__index_check(index_as_string)
        for i, obj in enumerate(self.items):
            if i == index_to_remove:
                if obj.is_done:
                    return ("Item: " + "'" + obj.description + "'" + " has been done already!")
                obj.mark_as_done()
                return ("Item: " + "'" + obj.description + "'" + " marked as done")
            
    def mark_item_as_pending(self, user_input):
        index_as_string = user_input[8:].strip()
        index_to_remove = self.__index_check(index_as_string)
        for i, obj in enumerate(self.items):
            if i == index_to_remove:
                if not obj.is_done:
                    return ("Item: " + "'" + obj.description + "'" + " is already pending!")
                obj.mark_as_pending()
                return ("Item: " + "'" + obj.description + "'" + " marked as pending")
    
    def __index_check(self, string):
        try:
            index = int(string.strip())
        except Exception:
            raise ValueError("'" + string + "'" + ' is not a number')
        if index < 1:
            raise ValueError('Index must be greater than 0')
        try:
            if self.items[index - 1]:
                return index - 1
        except IndexError:
            raise ValueError('No item at index ' + string)
            
    def delete_item(self, user_input):
        try:
            s = int(user_input[7:].strip())
            if s == 0:
                raise IndexError
            deleted_item = self.items.pop(s-1)
            return ("Task: " + "'" + deleted_item.description + "'" + " deleted from the list")
        except IndexError:
            raise IndexError("There is no list item at the number you typed!")
        except ValueError:
            raise ValueError("Only integers accepted as input")
            
    def get_current_progress(self):
        status = {'Todo': 0, 'Deadline': 0}
        tFlag = 0
        dFlag = 0
        for obj in self.items:
            if isinstance(obj, Deadline) and dFlag == 0:
                status['Deadline'] = obj.progress
                dFlag = 1
            elif tFlag == 0:
                status['Todo'] = obj.progress
                tFlag = 1
        return(">>> Progress for this session: todos {} deadlines {}".format(status['Todo'], status['Deadline']))
        
    def execute_command(self, command):
        if command == 'help':
            return self.get_help()
        elif command == 'progress':
            return self.get_current_progress()
        elif command.startswith('todo'):
            return self.add_item(command)
        elif command.startswith('deadline'):
            return self.add_deadline_item(command)
        elif command.startswith('done'):
            return self.mark_item_as_done(command)
        elif command.startswith('pending'):
            return self.mark_item_as_pending(command)
        elif command.startswith('delete'):
            return self.delete_item(command)
        else:
            raise Exception('Command not recognized')


GUI(TaskManager()).start()

In [ ]:
import unittest

def add_item(user_input):
        command_parts = user_input.strip().split(' ', 1)
        try:
            self.items.append(ToDo(command_parts[1], False))
            return ("New item: " + "'" + command_parts[1] + "'" + " added")
        except IndexError:
                raise IndexError("INPUT: todo \"task\"")
            
class TestSearch(unittest.TestCase):
    
    def test_add_item(self):
        
                
    def test_add_deadline_item(self):
        
                

# activate the test runner
if __name__ == '__main__':
    unittest.main()

### More flexibility in searching for items

Provide the ability to find tasks based on a search criterion e.g., find items that contains the provided keyword in their description.

### Mass operations

Provide a way to perform tasks on multiple items e.g., delete some specific items in one go.

### Unscheduled tasks with a fixed duration

Provide support for managing tasks that takes a fixed amount of time but does not have a fixed start/end time e.g., reading the sales report (needs 2 hours).

In [79]:
import unittest

def get_first_name(name):
    """Return the first part of the parameter 'name'"""
    return name.split()[0]


def is_same_person(person, keyword):
    """Return True if the parameter 'person' (type: dictionary)
    contains a key 'name' whose value contains the 
    parameter 'keyword' (type: string)
    e.g., 
    * is_same_person({'name': 'jackie'}, 'jack') returns True
    * is_same_person({'name': 'jackie'}, 'jackie-chan') returns False
    """
    return keyword in person['name']

class TestSearch(unittest.TestCase):

    def test_is_same_person(self):
        jack = {'name':'jack'}
        self.assertTrue(search.is_same_person(jack, 'jack'))
        self.assertTrue(search.is_same_person(jack, 'ack'))
        self.assertTrue(search.is_same_person(jack, 'ac'))
        self.assertTrue(search.is_same_person(jack, 'j'))
        self.assertTrue(search.is_same_person(jack, 'k'))
        self.assertFalse(search.is_same_person(jack, 'jackie'))
        self.assertFalse(search.is_same_person(jack, 'blackjack'))
        self.assertFalse(search.is_same_person({'name': 'x', 'other': 'jack'}, 'jack'))
        with self.assertRaises(KeyError):
            search.is_same_person({}, 'jack')

    def test_get_first_name(self):
        self.assertEqual(search.get_first_name('Amy'), 'Amy')
        self.assertEqual(search.get_first_name('Amy Bernice'), 'Amy')
        self.assertEqual(search.get_first_name('Amy-Bernice'), 'Amy-Bernice')
        with self.assertRaises(IndexError):
            search.get_first_name('')

# activate the test runner
if __name__ == '__main__':
    unittest.main()

E
ERROR: C:\Users\itsda\AppData\Roaming\jupyter\runtime\kernel-5e9e3dd5-3c3a-4228-8128-6c3dc690cc5b (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\itsda\AppData\Roaming\jupyter\runtime\kernel-5e9e3dd5-3c3a-4228-8128-6c3dc690cc5b'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

C:\Users\itsda\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
